# Hartree-Fock (SCF) Procedure

#### What are we trying to solve here?

$$ \hat{H}\psi = E\psi $$

where $\psi$ is the __electronic__ wave function of the system.

After BO approximation, the molecular Hamiltonian looks like:

$$ \hat{H} = \hat{T}_{e} + \hat{V}_{eN} + \color{red}{\hat{V}_{NN}} + \hat{V}_{ee} $$

where $\color{red}{\hat{V}_{NN}}$ is a constant.

Let's consider Helium hydride (HeH$^+$) in minimal STO-3G basis.

Why Helium hydride? Because $H_2$ is boring!!!

In [ ]:
!cat geom.xyz

## Important: We skip through a lot of nasty details such as guess wave function, explicitly computing the integrals, etc.

### This is so that we concentrate on the SCF cycle itself.

## Step 1: Compute the core Hamiltonian

### Step 1a: Compute $ V_{NN} $

In [ ]:
import re, sys, numpy, parse

a = parse.parse_file("geom.xyz")

num_atoms = 2
N_elec = 2
E_nn = 0.0
z1, z2 = 2.0, 1.0
r1 = numpy.array((float(a[2][1]),float(a[2][2]),float(a[2][3])))
r2 = numpy.array((float(a[3][1]),float(a[3][2]),float(a[3][3])))
r12 = numpy.linalg.norm(r1-r2)*1.889725989
E_nn += z1*z2/r12

E_nn

### Step 1b: Compute one-electron integrals (or read precomputed ones)

In [ ]:
overlap = numpy.genfromtxt('./overlap.dat')
T_e = numpy.genfromtxt('./T_e.dat')
V_eN = numpy.genfromtxt('./V_eN.dat')

In [ ]:
# Sample: Let's look at the Overlap integral.
# 1st and 2nd column belong to basis functions. Third column is the computed overlap value.
# Overlap matrix is a square symmetric matrix with all the diagonal elements equal to 1. 
# Here (and in real QM codes) it is represented as a triangular matrix. 

!cat overlap.dat

In [ ]:
N_basis = 2  # Minimal basis - STO-3G

S, Te, VeN = numpy.zeros((N_basis, N_basis)), numpy.zeros((N_basis, N_basis)), numpy.zeros((N_basis, N_basis))

In [ ]:
for i in range(0,N_basis):
    for j in range(0,i+1):
        S[i][j] = overlap[i+j][2]
        S[j][i] = S[i][j]
        #  print("{0},{1},{2}".format(i,j,S[i][j]))

for i in range(0,N_basis):
    for j in range(0,i+1):
        VeN[i][j] = V_eN[i+j][2]
        VeN[j][i] = VeN[i][j]

for i in range(0,N_basis):
    for j in range(0,i+1):
        Te[i][j] = T_e[i+j][2]
        Te[j][i] = Te[i][j]

#### Recap: How are these integrals computed? 

$$ S_{\mu\nu} = \int \phi_{\mu}(r)\phi_{\nu}(r)dr $$

$$ T_{\mu\nu} = \int \phi_{\mu}(r) \left (-\frac{1}{2} \nabla_r^2  \right ) \phi_{\nu}(r)dr $$

$$ V_{\mu\nu} = \int \phi_{\mu}(r) \left (- \sum_{A}^{N}\frac{Z}{r_A} \right ) \phi_{\nu}(r)dr $$ 

Let's see how these matrices look like.

In [ ]:
import pandas as pd
pd.DataFrame(S)  # Print the S Matrix in a readable form

In [ ]:
pd.DataFrame(Te)

In [ ]:
pd.DataFrame(VeN)

### Step 1c: Form the core Hamiltonian

$$ H_{\mu\nu}^{core} = T_{\mu\nu} + V_{\mu\nu} $$

In [ ]:
core_H = VeN + Te
pd.DataFrame(core_H)

### Step 1d: Form 2-electron index (using permutational symmetry)

In [ ]:
V_ee = numpy.genfromtxt('./V_ee.dat')

def eint(a,b,c,d):  
    if a > b: ab = a*(a+1)/2 + b  
    else: ab = b*(b+1)/2 + a  
    if c > d: cd = c*(c+1)/2 + d  
    else: cd = d*(d+1)/2 + c  
    if ab > cd: abcd = ab*(ab+1)/2 + cd  
    else: abcd = cd*(cd+1)/2 + ab  
    return abcd

ee_dict = {eint(row[0],row[1],row[2],row[3]) : row[4] for row in V_ee}
#ee_dict

### Step 2: Build the Orthogonalization matrix

$$ SL_S = L_S\epsilon_S $$

where $\epsilon$ is the diagonal matrix of eigenvalues and $ L_S $ is the matrix of eigenvectors.

$$ S^{-1/2} = L_{S} \epsilon^{-1/2} L_S^T $$

which is the symmetric orthogonalization matrix.

In [ ]:
S_eigval, S_eigvec = numpy.linalg.eig(S)

pd.DataFrame(S_eigval)

In [ ]:
S_eigval_minhalf = numpy.diag(S_eigval**(-0.5))
pd.DataFrame(S_eigval_minhalf)

In [ ]:
# S_minhalf = numpy.dot(S_eigvec,numpy.dot(S_eigval_minhalf,numpy.transpose(S_eigvec))) 
# pd.DataFrame(S_minhalf)

In [ ]:
S_mh = S_eigvec @ S_eigval_minhalf @ S_eigvec.T
pd.DataFrame(S_mh)

### Step 3: Main SCF Loop

Initial (guess) Fock matrix (in the orthonormal AO basis) using the core Hamiltonian as a guess:

$$ F_0 = ({S}^{-1/2})^T H^{core}S^{-1/2} $$

Diagonalize the Fock matrix:

$$ F_0 C_0^\prime = C_0^\prime \epsilon_0 $$

Now, transform the eigenvectors into the original (non-orthogonal) AO basis:

$$ C_0 = S^{-1/2} C_0^\prime $$



In [ ]:
P = numpy.zeros((N_basis, N_basis))   # The density matrix - set to zero initially.

# We do not explicitly work with the G-pqrs Matrix! We compute all the
# 2-e integrals on the fly, or read from file...

# G = numpy.zeros((N_basis, N_basis))   # Initialize the G-monster!


# NOTE: Read the definitions AFTER going through the main SCF cycle (last few lines of this block)


def deltap(P,OLDP, dim):  
    DELTA = 0.0e0  
    for i in range(0,dim):  
        for j in range(0,dim):  
            DELTA = DELTA+((P[i,j]-OLDP[i,j])**2)  
    DELTA = (DELTA/4)**(0.5)  
    return DELTA

def currentenergy(P,Hcore,F,dim):  
    EN = 0.0e0  
    for mu in range(0,dim):  
        for nu in range(0,dim):  
            EN = EN + 0.5*P[mu,nu]*(Hcore[mu,nu] + F[mu,nu])  
    return EN

def makedensity(C,P,dim,Nelec):  
    OLDP = numpy.zeros((dim,dim))  
    for mu in range(0,dim):  
        for nu in range(0,dim):  
            OLDP[mu,nu] = P[mu,nu]
            P[mu,nu] = 0.0e0
            for m in range(0,Nelec//2):  
                P[mu,nu] = P[mu,nu] + 2*C[mu,m]*C[nu,m]  
    return P, OLDP

def tei(a,b,c,d):  
    return ee_dict.get(eint(a,b,c,d),0.0)

def diagonalize(M):  
    e,Cprime = numpy.linalg.eigh(M)  
    #e=np.diag(e)  
    C = numpy.dot(S_minhalf,Cprime)  
    return e,C

def make_fock(Hcore,P,dim):  
    F = numpy.zeros((dim,dim))  
    for i in range(0,dim):  
        for j in range(0,dim):  
            F[i,j] = Hcore[i,j]  
            for k in range(0,dim):  
                for l in range(0,dim):  
                    F[i,j] = F[i,j] + P[k,l]*(tei(i+1,j+1,k+1,l+1)-0.5e0*tei(i+1,k+1,j+1,l+1))  
    return F

def fprime(X,F):  
    return numpy.dot(numpy.transpose(X),numpy.dot(F,X))


## MAIN SCF CYCLE !!!

for i in range(1, 11):
    print ("\n\nSCF Cycle ", i)
    F = make_fock(core_H, P, N_basis)
    print ("\n\nFock Matrix :\n", F)
    
    Fprime = fprime(S_minhalf,F)
    print ("\n\nF-Prime :\n", Fprime)
    
    E,Cprime = numpy.linalg.eigh(Fprime)
    # C = numpy.dot(S_minhalf,Cprime)
    C = S_minhalf @ Cprime
    E,C = diagonalize(Fprime)
    print ("\n\nC-Matrix :\n", C)
    print ("\n\nE-vector :\n", E)
    
    P,OLDP = makedensity(C,P,N_basis,N_elec)
    DELTA = deltap(P,OLDP, N_basis)
    print ("\n\nDensity Matrix :\n", P)
    
    EN = currentenergy(P,core_H,F,N_basis)
    print ("\n\nCurrent Energy = ", EN + E_nn)
    i += 1

print ("\n\nEnd of %i SCF iterations. Final RHF energy = %9.8f"%(i-1, EN+E_nn))


### Now, what can be done to improve the SCF code?

Question: Why are we setting the total number of SCF cycles (= 10) in advance?

Isn't it wrong to assume that the SCF converges in a set number of cycles? Instead:

* We can set an energy change threshold

* We can set a density change threshold

* We can set a density as well as energy change threshold, which is done in most software packages

In [ ]:
delta_E = 1.00                        # Change in energy. Start with a high number.
threshold = 0.00000001                # Energy threshold = 10^-8
energies = [0.50]                     # Set to a large number so that delta_E is high to start with...
P = numpy.zeros((N_basis, N_basis))   # The density matrix - set to zero initially.
i = 1

while delta_E > threshold:
    print ("\n\nSCF Cycle ", i)
    F = make_fock(core_H, P, N_basis)
#    print "\n\nFock Matrix :\n", F
    Fprime = fprime(S_minhalf,F)
#    print "\n\nF-Prime :\n", Fprime
    E,Cprime = numpy.linalg.eigh(Fprime)
    C = numpy.dot(S_minhalf,Cprime)
    E,C = diagonalize(Fprime)
#    print "\n\nC-Matrix :\n", C
    P,OLDP = makedensity(C,P,N_basis,N_elec)
    DELTA = deltap(P,OLDP, N_basis)
#    print "\n\nDensity Matrix :\n", P
    EN = currentenergy(P,core_H,F,N_basis)
    energies.append(EN)
    delta_E = abs(energies[-1] - energies[-2])
    print ("\n\nCurrent Energy = ", EN + E_nn)
    i += 1

print ("\n\nConvergence criterion reached. Final RHF energy =", EN+E_nn)


__QUIZ: The LUMO energy is negative. Does it mean anything?__
Hint: We are looking at (HeH+) system.

In [ ]:
pd.DataFrame(E)


## Moving on to computing integrals explicitly:

What does a 3D gaussian function look like?

$$ G_{ijk}(r,\alpha,A) = x_A^i y_A^j z_A^k exp(-\alpha r_A^2) $$

with orbital exponent α, electronic coordinates r, origin A, and $r_A = r - A $.

Also, i, j, k are angular quantum numbers (i = 0 is s-type, i = 1 is p-type, etc.)

They are separable in such a way that 

$$ G_{ijk} (r,\alpha, A) = G_i(x,\alpha, A_x) G_j(y,\alpha, A_y) G_k(z,\alpha, A_z) $$

where the 1-D gaussian is given by:

$$ G_i(x,\alpha, A_x) = (x-A_x)^i exp(-\alpha(x-A_x)^2) $$

Let's look at a very basic case of evaluating the overlap integral between two non-contracted spherical gaussian functions.



In [ ]:
def eval_gaussian(Qx,a,b):
    q = a*b/(a+b)
    return numpy.exp(-q*Qx*Qx)

def overlap(a,A,b,B):
    ''' Evaluates overlap integral between two Gaussians
        Returns a float.
        a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
        b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
        A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
        B:    list containing origin of Gaussian 'b'
    '''
    n = 0.458593068985           # Normalization factor
    S1 = eval_gaussian(A[0]-B[0],a,b) # X
    S2 = eval_gaussian(A[1]-B[1],a,b) # Y
    S3 = eval_gaussian(A[2]-B[2],a,b) # Z
    return S1*S2*S3*numpy.power(numpy.pi/(n*(a+b)),1.5)

## Now let's try to compute the integrals ourselve. 

Let's start with overlap integral. 
For simplicity, consider $H_2$ molecule in STO-3G basis - so each hydrogen contains a single 1s orbital. 

We can use the STO-3G basis for hydrogen (you can grab it from https://bse.pnl.gov/bse/portal).

$\alpha$ = 3.42525091

Let us center Hydrogens (and 1s gaussians) at 0.0 and 0.74 respectively.

In [ ]:
a = 3.42525091
b = 3.42525091
i = [0.0,0.0,0.0]
j = [0.74,0.0,0.0]


overlap(a, i, b, j)

What if you consider the overlap between two gaussians centered around the same point?

In [ ]:
a = 3.42525091
b = 3.42525091
i = [0.0,0.0,0.0]
j = [0.74,0.0,0.0]

overlap(a, i, b, i)

In [ ]:
S = [[0.0, 0.0], [0.0, 0.0]]
for x in range(2):
    S[0][0] = overlap(a, i, b, i)
    S[0][1] = overlap(a, i, b, j)
    S[1][0] = overlap(a, j, b, i) # Alternately, can use only upper/lower triangle of the overlap matrix.
    S[1][1] = overlap(a, j, b, j)
    
pd.DataFrame(S)   # Print the Overlap matrix

### Formulas for other integrals between 1s gaussians are provided in Appendix A of Szabo & Ostlund book. 
(Formulas A.11, A.33, A.41).

Note that electron-nuclear attraction and electron repulsion integrals are computed using error function err. Eff is implemented in scipy.

Please write a code to compute all these integrals

... and compute the HF energy of $H_2$

Now you can compute HF energy of $H_2$ at any bond distance. --- This is an optional step, you can skip it if busy... 

In [ ]:
# example of calling erf()
# from scipy import special
# y = special.erf(x)
